Django进阶实战10
====
**模板的继承 inheritance **

Django模板引擎中最强大（也是最复杂的）的部分是模板继承。模板继承允许您构建一个基础“骨架”模板，其中包含站点的所有常见元素，并定义子模板可以覆盖的块。

通过以示例开头，最容易理解模板继承：

```
<!DOCTYPE html>
<html lang="en">
<head>
    <link rel="stylesheet" href="style.css" />
    <title>{% block title %}My amazing site{% endblock %}</title>
</head>

<body>
    <div id="sidebar">
        {% block sidebar %}
        <ul>
            <li><a href="/">Home</a></li>
            <li><a href="/blog/">Blog</a></li>
        </ul>
        {% endblock %}
    </div>

    <div id="content">
        {% block content %}{% endblock %}
    </div>
</body>
</html>
```

此模板（我们将其命名为 base.html ）定义了一个简单的HTML框架文档，可以将其用于简单的双列页面。

“子”模板的工作是用内容填充**空块**。

在此示例中，块标记定义了子模板可以填充的三个块。

- {% block title %}
- {% block sidebar %}
- {% block content %}

所有块标记的作用是告诉模板引擎子模板可以覆盖模板的这些部分。

子模板可能如下所示：

```
{% extends "base.html" %}

{% block title %}My amazing blog{% endblock %}

{% block content %}
{% for entry in blog_entries %}
    <h2>{{ entry.title }}</h2>
    <p>{{ entry.body }}</p>
{% endfor %}
{% endblock %}
```

extends 标签是这里的关键。它告诉模板引擎该模板“扩展”另一个模板。当模板系统评估此模板时，首先它找到父模板 - 在本例中为“base.html”。

此时，模板引擎会注意到base.html中的三个块标记，并逐一查找子模板中的块标记，在子模板中找到与父模板中同名的块时，将用子模板的这个块内容替换父模板中对应的块内容。

父模板中的块在子模板中没有同名的块，继承父模板原块的内容。本例中：

- {% block title %} # 子模板中有同名的块，将被替换。
- {% block sidebar %} #子模板中没有同名的块，继承父模板块的内容。
- {% block content %} #子模板中有同名的块，也将被替换。

根据blog_entries的值，输出可能如下所示：

```
<!DOCTYPE html>
<html lang="en">
<head>
    <link rel="stylesheet" href="style.css" />
    <title>My amazing blog</title>
</head>

<body>
    <div id="sidebar">
        <ul>
            <li><a href="/">Home</a></li>
            <li><a href="/blog/">Blog</a></li>
        </ul>
    </div>

    <div id="content">
        <h2>Entry one</h2>
        <p>This is my first entry.</p>

        <h2>Entry two</h2>
        <p>This is my second entry.</p>
    </div>
</body>
</html>
```

请注意，由于子模板未定义侧边栏块，因此将使用父模板中的值。父模板中{％ block ％}标记内的内容始终用作后备。

您可以根据需要使用尽可能多的继承级别。使用继承的一种常见方法是下述的三级继承：

1. 创建一个base.html模板，其中包含您网站的主要外观。
2. 为网站的每个“部分”创建一个base_SECTIONNAME.html模板。例如，base_news.html，base_sports.html。 些模板都扩展了base.html，并包含特定于部分的样式/设计。
3. 为每种类型的页面创建单独的模板，例如新闻文章或博客条目。 这些模板扩展了相应的部分模板。

这种方法可以最大化代码重用，并且可以轻松地将项目添加到共享内容区域，例如部分范围的导航。

以下是使用继承的一些注意事项：

- 如果在模板中使用{％ extends ％}，则它必须是该模板中的第一个模板标记。否则，模板继承将不起作用。

- 基本模板中的{％ block ％}标记越多越好。请记住，子模板不必定义所有父块，因此您可以在多个块中填写合理的默认值，然后仅定义稍后需要的块。最好有更多的钩子而不是更少的钩子。

- 如果您发现自己在许多模板中复制了内容，则可能意味着您应该将该内容移动到父模板中的{％ block ％}。

- 如果您需要从父模板获取块的内容，{{ block.super }}变量将起作用。如果要添加父块的内容而不是完全覆盖它，这将非常有用。使用{{ block.super }}插入的数据不会自动转义（请参阅下一节），因为必要时，它已在父模板中转义。

- 使用模板标记作为语法在{％ block ％}之外创建的变量不能在块内使用。例如，此模板不呈现任何内容：

```
{% trans "Title" as title %}
{% block content %}{{ title }}{% endblock %}
```

为了提高代码的可读性，可以为{％ endblock ％}标记指定名称。 例如：

```
{% block content %}
...
{% endblock content %}
```
在较大的模板中，这样做可以帮助查看正在关闭的 {％ block ％} 标记。

最后，请注意，您无法在同一模板中定义多个具有相同名称的块标记。 存在这种限制是因为块标签在“两个”方向上工作。 也就是说，块标记不仅提供填充孔 - 它还定义填充父级孔的内容。 如果模板中有两个类似命名的块标记，则该模板的父级将不知道要使用哪个块的内容。